In [1]:
import pandas as pd 

### Import numpy and set random seed for reproducability
import numpy as np

### Download the dataset using the given code
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
ds = fetch_openml("mnist_784", as_frame=False)
x, x_test, y, y_test = train_test_split(ds.data, ds.target, test_size=0.2, random_state=42)

In [2]:
### Get the max/min of x and x_test. Will proceed to convert to uint8
assert np.max(x) == 255
assert np.min(x) == 0
assert np.max(x_test) == 255
assert np.min(x_test) == 0

In [3]:
### Convert x/x_test to uint8
x = x.astype('uint8')
x_test = x_test.astype('uint8')

In [4]:
### Randomly subsample x to create dataset of size 10000 with 1000 samples per label (subsample y accordingly)
df = pd.DataFrame(x)
df['label'] = y

# Sample 1000 rows per class
df_sub = df.groupby('label', group_keys=False).apply(lambda g: g.sample(1000, random_state=42))
x = df_sub.drop(columns='label').to_numpy()
y = df_sub['label'].to_numpy().flatten()

/var/folders/qj/tspvs3x90hg30hgcctblgwfh0000gn/T/ipykernel_57278/1568625766.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sub = df.groupby('label', group_keys=False).apply(lambda g: g.sample(1000, random_state=42))


In [5]:
### Use train_test_split to split x and y into train and validation sets (80% train, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
from sklearn import svm
classifier = svm.SVC(C = 1.0, kernel = 'rbf', gamma = 'auto')

### Fit to training data
classifier.fit(x_train, y_train)

### Predict labels of training data using trained classifier
y_train_pred = classifier.predict(x_train)

### Predict labels of validation data using trained classifier
y_val_pred = classifier.predict(x_val)

### Run the classifier on x_test,y_test
y_test_pred = classifier.predict(x_test)

### Training Error
from sklearn.metrics import accuracy_score
training_accuracy_score = accuracy_score(y_train, y_train_pred)
train_error = np.mean(y_train_pred != y_train)
print("Training Accuracy: ", training_accuracy_score)
print("Training Error: ", train_error)

### Validation Error
validation_accuracy_score = accuracy_score(y_val, y_val_pred)
val_error = np.mean(y_val_pred != y_val)
print("Validation Accuracy: ", validation_accuracy_score)
print("Validation Error: ", val_error)

### Compute ratio of the number of support samples to the total number of training samples for classifier
num_support_vectors = len(classifier.support_)
total_training_samples = x_train.shape[0]
support_vector_ratio = num_support_vectors / total_training_samples
print("Support Vector Ratio: ", support_vector_ratio)

### Report the classification error for Test Data
test_accuracy_score = accuracy_score(y_test, y_test_pred)
test_error = np.mean(y_test_pred != y_test)
print("Test Accuracy: ", test_accuracy_score)
print("Test Error: ", test_error)

### Report the 10-class confusion matrix on the test data
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:\n", conf_matrix)

Training Accuracy:  1.0
Training Error:  0.0
Validation Accuracy:  0.0875
Validation Error:  0.9125
Support Vector Ratio:  1.0
Test Accuracy:  0.10142857142857142
Test Error:  0.8985714285714286
Confusion Matrix:
 [[   0    0    0    0    0    0    0    0    0 1343]
 [   0    0    0    0    0    0    0    0    0 1600]
 [   0    0    0    0    0    0    0    0    0 1380]
 [   0    0    0    0    0    0    0    0    0 1433]
 [   0    0    0    0    0    0    0    0    0 1295]
 [   0    0    0    0    0    0    0    0    0 1273]
 [   0    0    0    0    0    0    0    0    0 1396]
 [   0    0    0    0    0    0    0    0    0 1503]
 [   0    0    0    0    0    0    0    0    0 1357]
 [   0    0    0    0    0    0    0    0    0 1420]]
